In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-t60pz_dx/kobert-tokenizer_92826757948f4e2a845dad3ab52cbb19
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-t60pz_dx/kobert-tokenizer_92826757948f4e2a845dad3ab52cbb19
  Resolve

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Softmax
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [4]:
# 필요한 클래스

class BERTSentenceTransform:
    
    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab ##추가

    def __call__(self, line): 
        

        # 유니코드로 변환
        text_a = line[0]
        if self._pair:
            assert len(line) == 2  # 2개로 이뤄진문장 있을 수 있음3
            text_b = line[1]

        #tokens_a = self._tokenizer(text_a)   
        tokens_a = self._tokenizer.tokenize(text_a) 
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # `tokens_a` 와 `tokens_b` max_seq_length-3을 넘지 않도록 수정
            # [CLS], [SEP], [SEP]를 포함하므로 "- 3" 들어가야함
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # 문장 한 개일 경우 [CLS], [SEP] 두 개이므로 "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        #input_ids = tokens_a['input_ids']

        # 문장의 유효길이 valid_length는 input_ids의 길이. 그 이외는 0 패딩
        valid_length = len(input_ids)

        if self._pad:
            # 시퀀스 길이만큼 0 패딩
            padding_length = self._max_seq_length - valid_length
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')


#데이터셋 input_ids, valid_length, segment_ids 로
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=19,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [5]:
# GPU 사용
device = torch.device("cuda:0")

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

No GPU available, using the CPU instead.


In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

## 사용자 정의 토큰 추가
new_tokens = ['우울', '슬픔', '외로움', '분노', '무기력', '의욕상실', '식욕저하', '식욕증가', '감정조절이상',
              '불면', '초조', '피로', '죄책감', '집중력저하', '자신감 저하', '자존감저하', '절망', '자살', '불안']
tokenizer.add_tokens(new_tokens)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


19

In [7]:
model = torch.load(f'/content/drive/MyDrive/KOBERT/model/kobert_text_depress_model_test1.pt', map_location=torch.device('cpu'))
model.eval()
print(model)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8021, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

In [8]:
# 파라미터
max_len = 64
batch_size = 16
warmup_ratio = 0.1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# 사용자 입력 전처리

In [9]:
"""!pip install konlpy
!pip install git+https://github.com/ssut/py-hanspell.git"""

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-zol2_ucz
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-zol2_ucz
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done


In [10]:
"""from konlpy.tag import Kkma
from hanspell import spell_checker

def split_sentence(text):
    kkma = Kkma()
    sentences = kkma.sentences(text)  # 입력 문장을 문장 단위로 분리합니다.
    result = []

    for sentence in sentences:
        morphemes = kkma.pos(sentence)  # 형태소 분석을 수행하여 형태소와 품사를 추출합니다.
        temp = []
        for i in range(len(morphemes)):
            if 'ECS' in morphemes[i][1] or 'EF' in morphemes[i][1]:  # 형태소의 품사가 'ECS' 또는 'EF'인 경우 종결어미로 판단합니다.
                if temp:
                    temp.append(morphemes[i][0])  # 현재 형태소를 결과에 추가합니다.
                    result.append(' '.join(temp))  # 분리된 형태소를 공백으로 연결하여 문장으로 생성합니다.
                    temp = []  # 임시 리스트를 초기화합니다.
            else:
                temp.append(morphemes[i][0])  # 형태소가 종결어미가 아닌 경우 결과에 추가합니다.

        if temp:
            result.append(' '.join(temp))  # 문장이 종결어미로 끝나지 않았을 경우, 남은 형태소를 결과에 추가합니다.

    return result



# 재생성된 문장 맞춤법 교정
def correct_spelling(text_list):
    tmp_list = []
    for t in text_list :
        spelled_sent = spell_checker.check(t)
        checked_sent = spelled_sent.checked
        tmp_list.append(checked_sent)
    return tmp_list"""

# 사용자 입력을 분석하여 감정 상태를 추론하고, 감정에 대한 점수를 반환하는 함수.


In [28]:
"""
사용자 입력을 분석하여 감정 상태를 추론하고, 감정에 대한 점수를 반환하는 함수.
"""

def DepressAnalyze_fn(user_input):
    
    # 사용자 입력을 문장으로 분리
    correct_sentences = user_input.split('다 ')
    correct_sentences = [c + '다. ' for c in correct_sentences]
    """predict_sentences = split_sentence(user_input)
    correct_sentences = correct_spelling(predict_sentences)"""

    for correct_sentence in correct_sentences : print(correct_sentence,"  ", len(correct_sentence))

    # 사용자 데이터셋 생성
    user_dataset = [[correct_sentence, '0']  for correct_sentence in correct_sentences]
    
    # logits 리스트 초기화
    logits_list = []

    # BERT 데이터셋 준비
    user_dataset_ids = BERTDataset(user_dataset, 0, 1, tokenizer, vocab, max_len, True, False)
    user_dataset_dataloader = torch.utils.data.DataLoader(user_dataset_ids, batch_size=batch_size, num_workers=2)
    
    # 모델을 사용하여 예측값 계산
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(user_dataset_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length.to(device)

        output = model(token_ids, valid_length, segment_ids)

        # 출력값 저장
        for logits in output:
            logits = logits.detach().cpu().numpy()
            logits_list.append(logits)

    # 감정 사전
    emotion_dict = {0: '감정조절이상', 1: '불면', 2: '분노', 3: '불안', 4: '초조', 5: '슬픔', 6: '외로움', 7: '우울', 8: '의욕상실', 9: '무기력',
                    10: '자살', 11: '자존감저하', 12: '절망', 13: '죄책감', 14: '집중력저하', 15: '피로', 16: '식욕저하', 17: '식욕증가', 18: '일상'}

    depress_list = []

    # softmax 점수 계산
    for logits in logits_list:
        softmax = Softmax(dim=0)
        score = softmax(torch.tensor(logits))  # 음성과 함께 사용할 예정
        max_value, max_index = torch.max(score, dim=0)

        emotion = emotion_dict[int(max_index)]
        depress_list.append(emotion)
        
        print("emotion :", emotion,"일 확률", round(100*(max_value.item()), 2),"%")
        print("")

    # 이 부분 수정 보완하기
    if (depress_list.count('일상')) != len(depress_list):
        depress_value = round((len(depress_list) - depress_list.count('일상')) / len(depress_list), 2)
    else :
        depress_value = 0.0

    return {'depress_list' : depress_list, 'depress_value' : depress_value}


In [29]:
## test1. 우울 문장 있을 경우
user_input = "요즘 나의 인간관계가 복잡하다 친한 친구와의 관계가 어색해지고 가족들과의 대화도 어색해진다 내가 무슨 말을 해도 상대방이 이해하지 못하는 것 같아서 답답하다 나는 왜 이렇게 다른 사람과의 대화가 어려운 걸까 내가 잘못한 걸까요즘 스트레스가 많이 쌓여서 힘들다 일과 공부 때문에 시간이 부족하고 여러 가지 일들이 겹쳐져서 정말 지친다 어쩌면 좋을까 휴식을 취하면서 스트레스를 풀 수 있는 방법을 찾아봐야겠다 혹시 좋은 조언이 있을까"
depress = DepressAnalyze_fn(user_input)
print(depress)

요즘 나의 인간관계가 복잡하다.     18
친한 친구와의 관계가 어색해지고 가족들과의 대화도 어색해진다.     35
내가 무슨 말을 해도 상대방이 이해하지 못하는 것 같아서 답답하다.     38
나는 왜 이렇게 다른 사람과의 대화가 어려운 걸까 내가 잘못한 걸까요즘 스트레스가 많이 쌓여서 힘들다.     58
일과 공부 때문에 시간이 부족하고 여러 가지 일들이 겹쳐져서 정말 지친다.     42
어쩌면 좋을까 휴식을 취하면서 스트레스를 풀 수 있는 방법을 찾아봐야겠다.     42
혹시 좋은 조언이 있을까다.     16
emotion : 감정조절이상 일 확률 57.92 %

emotion : 감정조절이상 일 확률 67.57 %

emotion : 초조 일 확률 38.5 %

emotion : 감정조절이상 일 확률 93.46 %

emotion : 피로 일 확률 94.26 %

emotion : 일상 일 확률 99.83 %

emotion : 일상 일 확률 98.31 %

{'depress_list': ['감정조절이상', '감정조절이상', '초조', '감정조절이상', '피로', '일상', '일상'], 'depress_value': 0.71}


In [30]:
## test2. 우울 문장 없을 경우
user_input = "오늘 학교에 왔다  친구들이랑 재미있게 놀았다  저녁에는 게임하고 놀거다  너무너무 신난다  "

depress = DepressAnalyze_fn(user_input)
print(depress)

오늘 학교에 왔다.     11
 친구들이랑 재미있게 놀았다.     17
 저녁에는 게임하고 놀거다.     16
 너무너무 신난다.     11
 다.     4
emotion : 일상 일 확률 99.93 %

emotion : 일상 일 확률 99.93 %

emotion : 일상 일 확률 99.93 %

emotion : 일상 일 확률 99.93 %

emotion : 일상 일 확률 99.84 %

{'depress_list': ['일상', '일상', '일상', '일상', '일상'], 'depress_value': 0.0}


##Flask API 서버 실행

In [31]:
!pip install flask 
!pip install pyngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!pip install flask jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-06-15 02:03:52--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.3’

ngrok-stable-linux- 100%[===================>]  13.21M  57.5MB/s    in 0.2s    

2023-06-15 02:03:52 (57.5 MB/s) - ‘ngrok-stable-linux-amd64.tgz.3’ saved [13856790/13856790]

ngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
# ngrok 개인 인증 키
!ngrok authtoken '2R5SgEswEFcWgKkawrdvZd3O7Yc_7QyjWgTEwHfUfGaVGnLNS'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [39]:
!pip install flask_cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from flask import Flask, jsonify, request, render_template, make_response
from flask_cors import CORS
from pyngrok import ngrok
import logging

app = Flask(__name__)
CORS(app)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False
ngrok.kill() # 초기화

# Logging 설정
logging.basicConfig(level=logging.DEBUG)

@app.route('/depress', methods=['GET','POST'])

@app.route('/emotion', methods=['GET','POST'])
def get_depress():

    if request.method == 'POST':

        data = request.get_json()
        user_input = data['user_input']
        depress = DepressAnalyze_fn(user_input)  

        return jsonify(depress)

    else:
        response = make_response('This is a GET request.')
        response.headers['Cache-Control'] = 'no-cache, no-store, must-revalidate'
        return response

def start_ngrok():
    ngrok_tunnel_url = ngrok.connect(80)
    print('Public URL:', ngrok_tunnel_url)

if __name__ == '__main__':
    start_ngrok()
    app.run(port=80)

Public URL: NgrokTunnel: "https://1d0b-35-185-5-17.ngrok-free.app" -> "http://localhost:80"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:80
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Jun/2023 03:10:05] "GET / HTTP/1.1" 200 -


제발 마지막에 제발 되라 제발다.     19


INFO:werkzeug:127.0.0.1 - - [15/Jun/2023 03:16:24] "POST / HTTP/1.1" 200 -


emotion : 일상 일 확률 99.93 %



In [33]:
$.get('/get_depress_data', function(data) {
    var depress_list = data['depress_list']; // 텍스트 우울 분석 후 문장 별 감정 리스트
    var depress_value_txt = data['depress_value']; // 텍스트 우울 분석 후 문단 전체에서 우울한 문장의 비율 : 0~1 사이 값
    var depress_value_snd = //상욱이가준우울분석한 0~1값 데이터

    var states = ['감정조절이상', '불면', '분노', '불안', '초조', '슬픔', '외로움', '우울', '의욕상실', '무기력', '자살', '자존감저하', '절망', '죄책감', '집중력저하', '피로', '식욕저하', '식욕증가', '일상'];
    var count_dict = {};
    
    for (var i = 0; i < states.length; i++) {
        count_dict[states[i]] = 0;        // 우울 양상 종류 딕셔너리 0으로 초기화
    }

    for (var i = 0; i < depress_list.length; i++) {
        if (depress_list[i] != '일상') {
            count_dict[depress_list[i]] += 1;       //일상 제외 우울 양상이면 해당 밸류값 +1 (키 값 count하는것)
        }
    }

    var total_depress_value = (depress_value_txt * 0.8 + depress_value_snd * 0.2); // 가중치 조절 해보기!! (텍스트 8: 음성 2로 시작)

    // total_depress_value값이 임계치 이상이면 우울 주의라고 판단하고 
    // 우울감 중 어떤 양상을 많이 보였는지 카운트 및 차트로 표시,
    // 안정, 명상에 도움이 되는 음악 재생,
    // 제공되는 퀘스트도 하루 중 가장 많이 차지하는 감정과 우울양상에 관련된 것으로 주기 위해 측정

    if (total_depress_value >= 0.5) {
        var ctx = document.getElementById('myChart').getContext('2d');
        var myChart = new Chart(ctx, {
            type: 'pie',
            data: {
                labels: states.filter(state => state !== '일상' && count_dict[state] > 0),
                datasets: [{
                    data: states.filter(state => state !== '일상' && count_dict[state] > 0).map(state => count_dict[state]),
                    backgroundColor: fixed_colors
                }]
            },
            options: {
                responsive: true,
                title: {
                    display: true,
                    text: 'Depression Levels'
                }
            }
        });
    }
});


SyntaxError: ignored